# Policy Evaluation (Prediction)

The initial approximation, $v_0$, is chosen arbitrarily
(except that the terminal state, if any, must be given value 0), and each successive approximation is obtained by using the Bellman expectation equation for $v_{\pi}$ as an update rule:

\begin{equation*}
v_{k+1}(s) \doteq  {\mathbb{E}}_{\pi} [R_{t+1} + \gamma v_k(s')| S_t = s] = \sum_{a}^{} \pi(a|s) \sum_{s', r}^{} p(s', r \ |s, a) [r + \gamma v_k(s')]
\end{equation*}

for all $s \in S$. Clearly, $v_k = v_{\pi}$ is a fixed point for this update rule because the Bellman equation for $v_{\pi}$ assures us of equality in this case. Indeed, the sequence $v_k$  can be shown in general to converge to $v_{\pi}$ as $k \rightarrow \infty$ under the same conditions that guarantee the existence of $v_{\pi}$ . This algorithm is called iterative policy evaluation.

Formally, iterative policy evaluation converges only in the limit, but in practice it must be halted short of this. As soon as $\Delta = \max_{s \in S}|v_{k+1}(s)-v_k(s)|$ is less than threshold $\theta$ the evaluation terminates.

- Input $\pi$, the policy to be evaluated <br>
- a small threshold $\theta > 0$ determining accuracy of estimation Initialize $V(s)$, for all $s \in S+$, arbitrarily except that $V(\text{terminal}) = 0$ <br>
- Loop: <br>
    + $\Delta \leftarrow 0$
    + Loop for each$s \in S$
        + $v \leftarrow V(s)$
        + $V(s) \leftarrow \sum_{a}^{} \pi(a|s) \sum_{s', r}^{} p(s', r \ |s, a) [r + \gamma V(s')]$
        + $\Delta = max(\Delta, |v-V(s)|)$
<br>
- Until $\Delta > \theta$

### Example 4.1 Grid World

In [4]:
import numpy as np
np.set_printoptions(formatter={'float': lambda x: "{0:0.1f}".format(x)})

actions = [(0, -1), (0, 1), (1, 0), (-1, 0)]
gamma = 1
theta = 0.01
policy = 0.25

def p(state, action):
    
    next_state = (state[0] + action[0], state[1] + action[1])
    if ((next_state[0] >= 0 and next_state[0] < 4) and
        (next_state[1] >= 0 and next_state[1] < 4)):
        return next_state, -1
    return state, -1 # wall
    
v = np.zeros((4, 4))
while True:
    delta = 0 
    for i in range(4):
        for j in range(4):
            if (i,j) == (0,0) or (i,j) == (3,3):
                continue
            v_prev = v[i, j]
            tmp = 0
            for a in actions:
                n_s, r = p((i, j), a)
                tmp += policy * (r + gamma * v[n_s[0], n_s[1]])
            v[i, j] = tmp
            delta = max(delta, abs(v[i, j]-v_prev))
    if delta < theta:
        break
print(np.round(v))

[[0.0 -14.0 -20.0 -22.0]
 [-14.0 -18.0 -20.0 -20.0]
 [-20.0 -20.0 -18.0 -14.0]
 [-22.0 -20.0 -14.0 0.0]]


#### Exercise 4.1 In Example 4.1, if $\pi$ is the equiprobable random policy, what is $q_{\pi}(11,down)$? What is $q_{\pi}(7,down)$?

Answer : $r(s,a,s')=-1$ for all s,s' and actions, and this is an undiscounted episodic task. values of $v_{\infty}$ are from figure 4.2 in book.

\begin{align*}
q_{\pi}(s, a) = \sum_{s', r}^{} p(s',r{\space}|s, a)[r(s,a,s') + {\gamma}v_{\pi}(s')]
\end{align*}
\begin{align*}
q_{\pi}(11, \text{down}) = -1 + 1\times0 = -1
\end{align*}
\begin{align*}
q_{\pi}(7, \text{down}) = -1 + 1\times(-14) = -15
\end{align*}

#### Exercise 4.2 In Example 4.1, suppose a new state 15 is added to the gridworld just below state 13, and its actions, left, up, right, and down, take the agent to states 12, 13, 14, and 15, respectively. Assume that the transitions from the original states are unchanged. What, then, is $v_{\pi}(15)$ for the equiprobable random policy? Now suppose the dynamics of state 13 are also changed, such that action down from state 13 takes the agent to the new state 15. What is $v_{\pi}(15)$ for the equiprobable random policy in this case?

Answer:

- unchanged

\begin{align*}
v_0(15) = 0
\end{align*}
\begin{align*}
v_{1}(15) = 0.25 \times [(-21) + (-23) + (-15) + (-1)] = -15.0
\end{align*}
\begin{align*}
v_{2}(15) = 0.25 \times [(-21) + (-23) + (-15) + (-16)] = -18.75
\end{align*}
\begin{align*}
...
\end{align*}
\begin{align*}
v_{\infty}(15) = -20
\end{align*}

- 13 is also changing

\begin{align*}
v_0(15) = 0
\end{align*}
\begin{align*}
v_0(13) = -20
\end{align*}
\begin{align*}
v_{1}(15) = 0.25 \times [(-21) + (-23) + (-15) + (-1)] = -15
\end{align*}
\begin{align*}
v_{1}(13) = 0.25 \times [(-21) + (-23) + (-15) + (-16)] = -18.75
\end{align*}
\begin{align*}
v_{1}(15) = 0.25 \times [(-19.75) + (-23) + (-15) + (-16)] = -18.4375
\end{align*}
\begin{align*}
v_{1}(13) = 0.25 \times [(-19.75) + (-23) + (-15) + (-19.4375)] = -19.296875
\end{align*}
\begin{align*}
...
\end{align*}
\begin{align*}
v_{\infty}(13) = -19.5
\end{align*}
\begin{align*}
v_{\infty}(15) = -19.5 
\end{align*}

#### Exercise 4.3 What are the equations analogous to (4.3), (4.4), and (4.5) for the action- valuefunction $q_{\pi}$ and its successive approximation by a sequence of functions $q_0$,$q_1$,$q_2$,...?

Answer:

\begin{align*}
q_{k+1}(s, a) = \sum_{s', r}^{} p(s',r{\space}|s, a)[r(s,a,s') + {\gamma}\sum_{a'}^{}\pi(a|s)q_{k}(s', a')]
\end{align*}

# Policy Improvement

Now that we know $v_{\pi}$, let's try to find better policies. We can choose an action a in some state s that is not in our policy and thereafter following the existing policy, $\pi$ (we call this new policy $\pi'$). If this choice is better than following $\pi$ all the  time $q_{\pi}(s,\pi'(s)) \ge v_{\pi}(s)$ than we can say that this new policy obtain greater or equal expected return from all states $s \in S$. In other word $v_{\pi'}(s) \ge v_{\pi}(s)$.

So far we have seen how, given a policy and its value function, we can easily evaluate a change in the policy at a single state to a particular action. It is a natural extension to consider changes at all states and to all possible actions, selecting at each state the action that appears best according to $q_{\pi}(s,a)$. In other words, to consider the new greedy policy, $\pi'$, given by

\begin{align*}
\pi'(s) = \text{argmax}_a q_{\pi}(s, a)
\end{align*}

The greedy policy takes the action that looks best in the short term—after one step of lookahead—according to $v_{\pi}$. so we know that it is as good as, or better than, the original policy. The process of making a new policy that improves on an original policy, by making it greedy with respect to the value function of
the original policy, is called policy improvement. If new policy is as good as old one then $v_{\pi}$ and $v_{\pi'}$ are both optimal policy $v_{*}$.

\begin{align*}
v_{*}(s) = v_{\pi}(s) = v_{\pi'}(s) = \max_a q_{\pi}(s, a)
\end{align*}

# Policy Iteration

Once a policy, $\pi$, has been improved using $v_{\pi}$ to yield a better policy, $\pi'$, we can then compute $v_{\pi'}$ and improve it again to yield an even better $\pi''$. Each policy is guaranteed to be a strict improvement over the previous one (unless it is already optimal). Because a finite MDP has only a finite number of policies, this process must converge to an optimal policy and optimal value function in a finite number of iterations. This way of finding an optimal policy is called policy iteration.

1. Initialization
   - $V(s) \in \mathbb{R}$ and $\pi(s) \in A(s)$ arbitrarily for all $s \in S$ <br>
<br>
2. Policy Evaluation
    - Loop: <br>
        + $\Delta \leftarrow 0$
        + Loop for each$s \in S$
            + $v \leftarrow V(s)$
            + $V(s) \leftarrow \sum_{a}^{} \pi(a|s) \sum_{s', r}^{} p(s', r \ |s, a) [r + \gamma V(s')]$
            + $\Delta = max(\Delta, |v-V(s)|)$
    <br>
    - Until $\Delta > \theta$
<br><br>
3. Policy Improvement
    - policy_stable $\leftarrow$ True
    - Loop for each$s \in S$
        + old_action $\leftarrow \pi(s)$
        + $\pi(s) \leftarrow \text{argmax}_a \sum_{s', r}^{} p(s',r{\space}|s, a)[r(s,a,s') + {\gamma}v_{k}(s')]$
        + if old_action $\neq \pi(s)$ then policy_stable $\leftarrow$ False
    - if policy_stable then stop and return $V \approx v_{*}$ and $\pi \approx \pi_{*}$ else goto 2

### Example 4.2 Jack’s Car Rental

In [1]:
import numpy as np
from math import factorial
from tqdm import tqdm
np.set_printoptions(formatter={'float': lambda x: "{0:0.1f}".format(x)})

actions = np.arange(-5, 6)
gamma = 0.9
theta = 1e-4
policy = np.zeros((21, 21), dtype=int)

cache = {}
def poisson(n, l):
    global cache
    if (n, l) not in cache:
        cache[(n,l)] = ((l**n)*(np.exp(-l)))/factorial(n)
    return cache[(n,l)]

#poisson for lambda=4 is mostly in range 0-9 / lambda=3 in range 0-7 / lambda=2 in range 0-5
outcomes = [(i, j, m, n) for i in range(9) for j in range(7) for m in range(7) for n in range(5)]
def EG(state, action, v):
    
    returns = -2 * abs(action)
    ret_fir, ret_sec = 3, 2
    for (rent_fir, rent_sec, ret_fir, ret_sec) in outcomes:
        new_state = (min(state[0] - action, 20), min(state[1] + action, 20))
        reward = 10*(min(new_state[0], rent_fir) + min(new_state[1], rent_sec))
        new_state = (new_state[0] - min(new_state[0], rent_fir), new_state[1] - min(new_state[1], rent_sec))
        prob = poisson(rent_fir, 4) * poisson(rent_sec, 3) * poisson(ret_fir, 3) * poisson(ret_sec, 2)
        new_state = (min(new_state[0] + ret_fir, 20), min(new_state[1] + ret_sec, 20))
        returns += prob * (reward + gamma * v[new_state[0], new_state[1]])

    return returns

value = np.zeros((21, 21))
states = [(i, j) for i in range(21) for j in range(21)]
iteration = 5
for i in tqdm(range(iteration)):

    while True:
        delta = 0
        for state in states:
            v_prev = v[state[0], state[1]]
            v[state[0], state[1]] = EG((state[0], state[1]), policy[state[0], state[1]], v)
            delta = max(delta, abs(v[state[0], state[1]]-v_prev))
        if delta < theta:
            break

    policy_stable = True
    policy_changes = 0
    for state in states:
        action_returns = []
        old_action = policy[state[0], state[1]]
        for action in actions:
            if (0 <= action <= state[0]) or (state[1] >= abs(action) > 0):
                action_returns.append(EG((state[0], state[1]), action, v))
            else:
                action_returns.append(-np.Infinity)
        policy[state[0], state[1]] = actions[np.argmax(action_returns)]
        if old_action != policy[state[0], state[1]]:
            policy_stable = False
            policy_changes += 1
    
    if policy_stable:
        break
    print('iteration#{} -> {} policies changed.'.format(i+1, policy_changes))
    

print('-'*10)
print('V*')
print(v)
print('-'*10)
print('Policy')
print(policy)

 20%|██        | 1/5 [02:34<10:17, 154.26s/it]

iteration#1 -> 215 policies changed.


 40%|████      | 2/5 [06:21<09:32, 190.86s/it]

iteration#2 -> 147 policies changed.


 60%|██████    | 3/5 [09:16<06:10, 185.40s/it]

iteration#3 -> 9 policies changed.
----------
V*
[[132.1 140.6 148.6 155.8 162.4 168.8 175.0 180.9 186.2 191.2 195.8 199.9
  203.8 207.2 210.3 213.1 215.6 217.9 219.9 221.8 223.5]
 [140.7 149.2 157.2 164.4 170.8 177.0 182.9 188.2 193.2 197.8 201.9 205.8
  209.2 212.3 215.1 217.8 220.1 222.3 224.2 225.9 227.5]
 [149.0 157.5 165.6 172.8 179.0 184.9 190.2 195.2 199.8 203.9 207.8 211.2
  214.3 217.1 219.8 222.1 224.3 226.2 227.9 229.5 230.9]
 [156.9 165.4 173.4 180.6 186.9 192.2 197.2 201.8 205.9 209.8 213.2 216.3
  219.1 221.8 224.1 226.3 228.2 229.9 231.5 232.9 234.2]
 [164.0 172.5 180.5 187.7 193.9 199.2 203.8 207.9 211.8 215.2 218.3 221.1
  223.8 226.1 228.3 230.2 231.9 233.5 234.9 236.2 237.3]
 [170.5 178.8 186.8 194.0 200.1 205.4 209.9 213.8 217.2 220.3 223.1 225.8
  228.1 230.3 232.2 233.9 235.5 236.9 238.2 239.3 240.3]
 [176.8 184.8 192.3 199.4 205.5 210.7 215.1 218.9 222.2 225.1 227.8 230.1
  232.3 234.2 235.9 237.5 238.9 240.2 241.3 242.3 243.2]
 [182.8 190.3 197.4 204.2 210.2 21

#### Exercise 4.4 The policy iteration algorithm on page 80 has a subtle bug in that it may never terminate if the policy continually switches between two or more policies that are equally good. This is ok for pedagogy, but not for actual use. Modify the pseudocode so that convergence is guaranteed.

Answer: We can check the sum of all state values that make the optimal policy, ($\sum_{s}^{} v_{\pi'}(s) = \sum_{s}^{}\max_a q_{\pi}(s, a)$) if this sum converges then we got our optimal policy. 

#### Exercise 4.5 How would policy iteration be defined for action values? Give a complete algorithm for computing q⇤, analogous to that on page 80 for computing v⇤. Please pay special attention to this exercise, because the ideas involved will be used throughout the rest of the book.

#### Exercise 4.6 Suppose you are restricted to considering only policies that are "-soft, meaning that the probability of selecting each action in each state, s, is at least "/|A(s)|. Describe qualitatively the changes that would be required in each of the steps 3, 2, and 1, in that order, of the policy iteration algorithm for v⇤ on page 80.

#### Exercise 4.7 (programming) Write a program for policy iteration and re-solve Jack’s car rental problem with the following changes. One of Jack’s employees at the first location rides a bus home each night and lives near the second location. She is happy to shuttle one car to the second location for free. Each additional car still costs \$2, as do all cars moved in the other direction. In addition, Jack has limited parking space at each location. If more than 10 cars are kept overnight at a location (after any moving of cars), then an additional cost of $4 must be incurred to use a second parking lot (independent of how many cars are kept there). These sorts of nonlinearities and arbitrary dynamics often occur in real problems and cannot easily be handled by optimization methods other than dynamic programming. To check your program, first replicate the results given for the original problem.

# Value Iteration

### Example 4.3 Gambler’s Problem

#### Exercise 4.8 Why does the optimal policy for the gambler’s problem have such a curious form? In particular, for capital of 50 it bets it all on one flip, but for capital of 51 it does not. Why is this a good policy?

#### Exercise 4.9 (programming) Implement value iteration for the gambler’s problem and solve it for ph = 0.25 and ph = 0.55. In programming, you may find it convenient to introduce two dummy states corresponding to termination with capital of 0 and 100, giving them values of 0 and 1 respectively. Show your results graphically, as in Figure 4.3. Are your results stable as ✓ ! 0?

#### Exercise 4.10 What is the analog of the value iteration update (4.10) for action values, qk+1(s, a)?

# Asynchronous Dynamic Programming

# Generalized Policy Iteration

# Efficiency of Dynamic Programming